这段代码展示了如何使用LlamaIndex库构建并更新Prompt模板。具体来说：

1. **定义Prompt模板**：通过`PromptTemplate`和`ChatPromptTemplate`创建用于文本生成的提示词。
2. **使用Prompt模板**：
   - 通过`qa_template.format()`和`chat_template.format(topic=topic)`插入上下文信息并生成相应的提示词字符串。
   - 使用`qa_template.format_messages()`和`chat_template.format_messages(topic=topic)`生成ChatMessage对象，模拟聊天场景。
3. **构建索引与查询引擎**：通过`VectorStoreIndex`从文档中构建向量存储索引，并设置LLM模型和嵌入模型。
4. **更新查询引擎的提示词**：将之前定义的文本问答模板应用到查询引擎上。

整个过程展示了如何在LlamaIndex中灵活地使用不同类型的Prompt模板来满足特定的应用场景需求。

In [1]:
from llama_index.core import PromptTemplate

template = (
    "我提供以下上下文信息\n"
    "-------------------\n"
    "{context_str}\n"
    "-------------------\n"
    "给定这个信息，请回答问题：{query_str}\n"
)

qa_template = PromptTemplate(template=template)

# 文本生成场景
context_str = "在阳光明媚的春日午后，小明和小华相约在公园的樱花树下野餐。铺开格子布，摆上三明治和果汁，两人边吃边聊，笑声不断。樱花随风飘落，像粉色的雪花，落在他们的头上和食物上，增添了几分浪漫。小明突然提议玩一个猜谜游戏，小华兴奋地接受了挑战，两人的友谊在这场智力角逐中更加深厚。"
query_str = "谁提出玩游戏的？"
prompt = qa_template.format(context_str=context_str, query_str=query_str)
print(prompt)

我提供以下上下文信息
-------------------
在阳光明媚的春日午后，小明和小华相约在公园的樱花树下野餐。铺开格子布，摆上三明治和果汁，两人边吃边聊，笑声不断。樱花随风飘落，像粉色的雪花，落在他们的头上和食物上，增添了几分浪漫。小明突然提议玩一个猜谜游戏，小华兴奋地接受了挑战，两人的友谊在这场智力角逐中更加深厚。
-------------------
给定这个信息，请回答问题：谁提出玩游戏的？



In [2]:
message=qa_template.format_messages(context_str=context_str, query_str=query_str)
print(message)

[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='我提供以下上下文信息\n-------------------\n在阳光明媚的春日午后，小明和小华相约在公园的樱花树下野餐。铺开格子布，摆上三明治和果汁，两人边吃边聊，笑声不断。樱花随风飘落，像粉色的雪花，落在他们的头上和食物上，增添了几分浪漫。小明突然提议玩一个猜谜游戏，小华兴奋地接受了挑战，两人的友谊在这场智力角逐中更加深厚。\n-------------------\n给定这个信息，请回答问题：谁提出玩游戏的？\n')])]


除了使用PromptTemplate外，类似的方法还可以创建ChatPromptTemplate   

In [3]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage,MessageRole

message_template=[
    ChatMessage(content="你是一个说书人",role=MessageRole.SYSTEM),
    ChatMessage(content="给我讲讲{topic}？",role=MessageRole.USER)
]

chat_template=ChatPromptTemplate(message_templates=message_template)

topic="三国演义的空城计"

# 用于文本生成 
prompt=chat_template.format(topic=topic)
print(prompt)

print('----------')

# 用于聊天
message=chat_template.format_messages(topic=topic)
print(message)

system: 你是一个说书人
user: 给我讲讲三国演义的空城计？
assistant: 
----------
[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='你是一个说书人')]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='给我讲讲三国演义的空城计？')])]


通过以上两种方式定义prompt，以下将说明如何将这些prompt用起来

In [4]:
# 构建索引及引擎 
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
documents = SimpleDirectoryReader("../../data/三国演义白话文/",recursive=True).load_data(show_progress=True)

# bge-base embedding model
base_url='http://localhost:11434'
Settings.embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url=base_url)

# ollama
Settings.llm = Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url=base_url)

index = VectorStoreIndex.from_documents(documents,show_progress=True)
query_engine = index.as_query_engine(response_mode='compact')

Loading files: 100%|██████████| 41/41 [00:00<00:00, 394.94file/s]
c:\Users\wushaogui\miniconda3\envs\langchian\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 127/127 [00:06<00:00, 18.39it/s]


In [5]:
# 获取引擎的当前prompt
cur_prompt=query_engine.get_prompts()
print(cur_prompt)

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x000002C450CB8160>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageR

In [6]:
# 更新prompt 
query_engine.update_prompts(
    {'response_synthesizer:text_qa_template':qa_template}
)

cur_prompt=query_engine.get_prompts()
print(cur_prompt)


{'response_synthesizer:text_qa_template': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='我提供以下上下文信息\n-------------------\n{context_str}\n-------------------\n给定这个信息，请回答问题：{query_str}\n'), 'response_synthesizer:refine_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.REFINE: 'refine'>}, template_vars=['query_str', 'existing_answer', 'context_msg'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.REFINE: 'refine'>}, template_vars=['query_str', 'existing_answer', 'context_msg'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="The original query is as follows: {query_str}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the ex

可以看到，prompt被更新，和langchian相比，其提示词没有那么抽象